In [1]:
import pandas as pd

In [6]:
# Change the cell format to 'YYYY-MM-DD' in Excel before loading the data.
df = pd.read_csv('FitNotes_Export_2025_Copy.csv')
df.head(5)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4120 entries, 0 to 4119
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           4120 non-null   object 
 1   Exercise       4120 non-null   object 
 2   Category       4120 non-null   object 
 3   Weight         4119 non-null   float64
 4   Weight Unit    4119 non-null   object 
 5   Reps           4119 non-null   float64
 6   Distance       1 non-null      float64
 7   Distance Unit  1 non-null      object 
 8   Time           1 non-null      object 
 9   Comment        600 non-null    object 
dtypes: float64(3), object(7)
memory usage: 322.0+ KB


In [7]:
df['Category'].unique().tolist()

['Legs',
 'Biceps',
 'Chest',
 'Triceps',
 'Back',
 'Shoulders',
 'Core',
 'Forearms',
 'Extras',
 'Dynamic',
 'Home Workouts']

In [8]:
# Deleteing useless columns.
df = df.drop(columns=['Distance', 'Distance Unit', 'Time', 'Comment'])

# Deleting records on basis of useless categories.
df = df.drop(df[df['Category'].isin(['Extras','Dynamic','Home Workouts'])].index)

df.info()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 4068 entries, 0 to 4114
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         4068 non-null   object 
 1   Exercise     4068 non-null   object 
 2   Category     4068 non-null   object 
 3   Weight       4068 non-null   float64
 4   Weight Unit  4068 non-null   object 
 5   Reps         4068 non-null   float64
dtypes: float64(2), object(4)
memory usage: 222.5+ KB


,Date,Exercise,Category,Weight,Weight Unit,Reps
0,2023-07-07,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0
1,2023-07-07,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0


In [9]:
df.isna().sum() # No NaNs that's a relief.

Date           0
Exercise       0
Category       0
Weight         0
Weight Unit    0
Reps           0
dtype: int64

In [10]:
# Standeredization of col names.
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
df.columns.tolist()

['date', 'exercise', 'category', 'weight', 'weight_unit', 'reps']

In [11]:
df.head(10)

,date,exercise,category,weight,weight_unit,reps
0,2023-07-07,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0
1,2023-07-07,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0
2,2023-07-07,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0
3,2023-07-07,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0
4,2023-07-07,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0
5,2023-07-07,Short Head - Db Curl(Twist),Biceps,10.0,kgs,7.0
6,2023-07-07,Short Head - Db Curl(Twist),Biceps,10.0,kgs,7.0
7,2023-07-07,Short Head - Db Curl(Twist),Biceps,10.0,kgs,7.0
8,2023-07-07,Short Head - Db Curl(Twist),Biceps,10.0,kgs,7.0
9,2023-07-08,Clavicle Head - Incline Bb Bench Press,Chest,12.5,kgs,8.0


# Normalizing the Original data

## Creating `d_muscle_group` table.

In [12]:
unique_categories = df['category'].unique()

gps = { 'muscle_group_key': list(range(1, len(unique_categories) + 1)),
        'muscle_group': unique_categories.tolist() }

muscle_gps = pd.DataFrame(gps)
muscle_gps.to_csv('d_muscle_gps.csv', index=False)
muscle_gps

,muscle_group_key,muscle_group
0,1,Legs
1,2,Biceps
2,3,Chest
3,4,Triceps
4,5,Back
5,6,Shoulders
6,7,Core
7,8,Forearms


## Extracting 'muscle_part' and 'exersice' from the `exercise` column of main df.

In [13]:
# Extracting unique 'category' and 'exercise' pairs from the main df.
unique_exercises = df.sort_values(by=['category', 'exercise'], ignore_index=True)[['category', 'exercise']]
unique_exercises = unique_exercises.drop_duplicates(keep='first', ignore_index=True)

for category in unique_exercises['category'].unique():
    print(unique_exercises.loc[unique_exercises['category'] == category].reset_index(drop=True))
    print()

   category                               exercise
0      Back                    Erectors - Deadlift
1      Back              Erectors - Hyperextension
2      Back               Lats - Assisted Chin Ups
3      Back               Lats - Machine Row Close
4      Back                       Lats - Pull Over
5      Back       Lats - Seated  Neutral Cable Row
6      Back                      Lats - TBar Close
7      Back               Lats - Under Grip Bb Row
8      Back              Lats - Undergrip pulldown
9      Back                       Traps - Bb Shrug
10     Back                       Traps - Db Shrug
11     Back                       Traps - Y Raises
12     Back     Upper Back -  Incline Dumbbell Row
13     Back          Upper Back - Assisted Pullups
14     Back       Upper Back - Close Grip Pulldown
15     Back                   Upper Back - Db Rows
16     Back        Upper Back - Low-high Cable Row
17     Back          Upper Back - Machine Row Wide
18     Back          Upper Back

# Fixs to be made at CSV source level
### Fixing data at CSV source to avoid extra competition/overhead in Python code

In [14]:
                                    #  ❗ Add muscle part in Biceps, chest and Forearms ❗ ←-----------|
                                    #                                                                  |
                                    #  ❗ Add muscle part for                       ❗               |
                                    #  ❗     - Seated Ez Bar Cable  Rows (Back)     ❗             |
                                    #  ❗     - Under-grip Cable Pushdown (Triceps) ❗             |
                                    #                                                              |
                                    #                                                             |   Will be solved here
                                    #  ❗       Standerdize muscle_part values      ❗          |
                                    #  Category 'Abs' should be 'Core'                          |
                            #        'Lower Abs', 'Obliques', 'Upper Abs',                     |
                            #        'Erectors', 'Lats', nan, 'Traps', 'Upper Back'           |
                            #        'EZ', 'Upper Chest',----------------------------------|
                            #        'Calves', 'Hamstrings', 'Inner Quads', 'Quads',
                            #                  'Hamstring',
                            #        'Front Delt', 'Front Side Delt', 'Lateral Delt',  'Rear Delt'
                            #        'Front Delts',                   'Lateral Delts', 'Rear Delts'
                            #        'Lateral Head', 'Long Head'
                            #                        'Long Head T'

In [15]:
# Extracting unique 'muscle_part' per muscle group from 'exercise' column.

# Splitting the 'exercise' column.
split_cols = unique_exercises['exercise'].str.split(' - ', n=1, expand=True)


unique_exercises['muscle_part'] = split_cols[0].where(unique_exercises['exercise'].str.contains(' - '))
unique_exercises['exercise_2'] = split_cols[1].fillna(split_cols[0])

# Populating `muscle_group_key` from muscle_gps table.

In [16]:
unique_exercises = unique_exercises.merge( muscle_gps,
                                           left_on='category',
                                           right_on='muscle_group',
                                           how='left' )
unique_exercises = unique_exercises.drop(columns=['muscle_group'])


unique_exercises.head(15)

,category,exercise,muscle_part,exercise_2,muscle_group_key
0,Back,Erectors - Deadlift,Erectors,Deadlift,5
1,Back,Erectors - Hyperextension,Erectors,Hyperextension,5
2,Back,Lats - Assisted Chin Ups,Lats,Assisted Chin Ups,5
3,Back,Lats - Machine Row Close,Lats,Machine Row Close,5
4,Back,Lats - Pull Over,Lats,Pull Over,5
5,Back,Lats - Seated Neutral Cable Row,Lats,Seated Neutral Cable Row,5
6,Back,Lats - TBar Close,Lats,TBar Close,5
7,Back,Lats - Under Grip Bb Row,Lats,Under Grip Bb Row,5
8,Back,Lats - Undergrip pulldown,Lats,Undergrip pulldown,5
9,Back,Traps - Bb Shrug,Traps,Bb Shrug,5


# Creating `exercise_key` and `muscle_part_key` column.

In [17]:
# Creating exercise_key column

# Create a sequential counter within each muscle_group_key
unique_exercises['exercise_key'] = (
    unique_exercises.groupby('muscle_group_key').cumcount() + 1
)

# Prefix with muscle_group_key
unique_exercises['exercise_key'] = (
    unique_exercises['muscle_group_key'].astype(str) +
    unique_exercises['exercise_key'].astype(str).str.zfill(2).str[-2:]
)

# Creating muscle_part_key column

# Create a unique rank per muscle_part within each muscle_group_key
unique_exercises['muscle_part_key'] = (
    unique_exercises.groupby('muscle_group_key')['muscle_part']
      .transform(lambda x: pd.factorize(x)[0] + 1)
      .astype(str)
)

# Prefix with muscle_group_key
unique_exercises['muscle_part_key'] = unique_exercises['muscle_group_key'].astype(str) + unique_exercises['muscle_part_key']


In [18]:
# Droppping redundent coluns.
unique_exercises = unique_exercises.drop(columns=['category', 'exercise'])
unique_exercises = unique_exercises.rename(columns={'exercise_2':'exercise'})

In [19]:
unique_exercises.head(10)

,muscle_part,exercise,muscle_group_key,exercise_key,muscle_part_key
0,Erectors,Deadlift,5,501,51
1,Erectors,Hyperextension,5,502,51
2,Lats,Assisted Chin Ups,5,503,52
3,Lats,Machine Row Close,5,504,52
4,Lats,Pull Over,5,505,52
5,Lats,Seated Neutral Cable Row,5,506,52
6,Lats,TBar Close,5,507,52
7,Lats,Under Grip Bb Row,5,508,52
8,Lats,Undergrip pulldown,5,509,52
9,Traps,Bb Shrug,5,510,53


# Creating `d_muscle_part` table

In [20]:
d_muscle_part = unique_exercises[['muscle_part_key', 'muscle_part', 'muscle_group_key']]
d_muscle_part = d_muscle_part.drop_duplicates(keep='first', ignore_index=True)
d_muscle_part.to_csv('d_muscle_part.csv', index=False)
d_muscle_part.head(5)

,muscle_part_key,muscle_part,muscle_group_key
0,51,Erectors,5
1,52,Lats,5
2,53,Traps,5
3,54,Upper Back,5
4,21,All Heads,2


# Creating `d_exercise` table.

In [23]:
d_exercise = unique_exercises[['exercise_key', 'exercise', 'muscle_part_key']]
d_exercise = d_exercise.drop_duplicates(keep='first', ignore_index=True)
d_exercise.to_csv('d_exercise.csv', index=False)
d_exercise.head(5)

,exercise_key,exercise,muscle_part_key
0,501,Deadlift,51
1,502,Hyperextension,51
2,503,Assisted Chin Ups,52
3,504,Machine Row Close,52
4,505,Pull Over,52


# Creating `d_workouts` tables.

In [32]:
# Copying og df.
df_new = df.copy()

## Creating date_key inplace of dates
df_new['date'] = df_new['date'].str.replace('-','')

# Imputing 'muscle_group_key' from muscle_gps table on 'category' column.
df_new = df_new.merge(muscle_gps, left_on='category', right_on='muscle_group', how='left')

# Dropping reduntent columns
df_new = df_new.drop(columns=['muscle_group'])

df_new.head(5)

,date,exercise,category,weight,weight_unit,reps,muscle_group_key
0,20230707,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0,1
1,20230707,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0,1
2,20230707,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0,1
3,20230707,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0,1
4,20230707,Quads - Leg Ext Machine,Legs,44.0,kgs,10.0,1


In [33]:
# Splitting column 'excercise'.
exercise_split = df_new['exercise'].str.split(' - ', n=1, expand=True)

# Adding the two parts of exercise column in df_new.
df_new['1'] = exercise_split[0]
df_new['2'] = exercise_split[1]

# Dropping reduntent columns
df_new = df_new.drop(columns=['exercise','category'])

In [34]:
# Imputing 'muscle_part_key' from d_muscle_part table on newly created column '1'.
df_new = df_new.merge(d_muscle_part, left_on='1', right_on='muscle_part', how='left')

# Imputing 'exercise_key' from d_exercie table on newly created column '2'.
df_new = df_new.merge(d_exercise, left_on='2', right_on='exercise', how='left')

In [35]:
# Dropping reduntent and duplicate columns to have some clarity on the final table.
df_new = df_new.drop(columns=['1','2','muscle_group_key_y','exercise','muscle_part_key_y'])

df_new.head(5)

,date,weight,weight_unit,reps,muscle_group_key_x,muscle_part_key_x,muscle_part,exercise_key
0,20230707,44.0,kgs,10.0,1,14,Quads,114
1,20230707,44.0,kgs,10.0,1,14,Quads,114
2,20230707,44.0,kgs,10.0,1,14,Quads,114
3,20230707,44.0,kgs,10.0,1,14,Quads,114
4,20230707,44.0,kgs,10.0,1,14,Quads,114


In [36]:
# Final d_workouts table.
d_workouts = df_new[['date', 'weight', 'weight_unit', 'reps', 'exercise_key']]

d_workouts = d_workouts.rename(columns={'date':'date_key'})
d_workouts.head(5)

,date_key,weight,weight_unit,reps,exercise_key
0,20230707,44.0,kgs,10.0,114
1,20230707,44.0,kgs,10.0,114
2,20230707,44.0,kgs,10.0,114
3,20230707,44.0,kgs,10.0,114
4,20230707,44.0,kgs,10.0,114


In [37]:
d_workouts.to_csv('d_workouts.csv', index=False)

# Loading all tables to MySQL database.

In [43]:
import pandas as pd
from sqlalchemy import create_engine

username = 'root'
password = input('Enter Password: ')
host = 'localhost'
port = '3306'
db_name = 'workout_project'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{db_name}")

# Dictionary mapping table names to DataFrames
tables = { 'd_workouts': d_workouts,
           'd_muscle_group': muscle_gps,
           'd_muscle_part': d_muscle_part,
           'd_exercie': d_exercise }

for name, df in tables.items():
    df.to_sql(name, engine, if_exists='replace', index=False)
    print(f'Table Name: {name}')
    print(pd.read_sql(f"SELECT * FROM {name} LIMIT 5\n", engine))
    print('\n')

Enter Password:  741


Table Name: d_workouts
   date_key  weight weight_unit  reps exercise_key
0  20230707    44.0         kgs  10.0          114
1  20230707    44.0         kgs  10.0          114
2  20230707    44.0         kgs  10.0          114
3  20230707    44.0         kgs  10.0          114
4  20230707    44.0         kgs  10.0          114


Table Name: d_muscle_group
   muscle_group_key muscle_group
0                 1         Legs
1                 2       Biceps
2                 3        Chest
3                 4      Triceps
4                 5         Back


Table Name: d_muscle_part
  muscle_part_key muscle_part  muscle_group_key
0              51    Erectors                 5
1              52        Lats                 5
2              53       Traps                 5
3              54  Upper Back                 5
4              21   All Heads                 2


Table Name: d_exercie
  exercise_key           exercise muscle_part_key
0          501           Deadlift              51
1   